In [1]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
from keras import layers
from tensorflow.keras.applications import ResNet50,MobileNet, DenseNet201, InceptionV3, NASNetLarge, InceptionResNetV2, NASNetMobile
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm
import tensorflow as tf
from keras import backend as K
import gc
from functools import partial
from sklearn import metrics
from collections import Counter
import json
import itertools


%matplotlib inline

ModuleNotFoundError: No module named 'keras'

In [ ]:
#Transfer 'jpg' images to an array IMG
def Dataset_loader(DIR, RESIZE, sigmaX=10):
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    for IMAGE_NAME in tqdm(os.listdir(DIR)):
        PATH = os.path.join(DIR,IMAGE_NAME)
        _, ftype = os.path.splitext(PATH)
        if ftype == ".png":
            img = read(PATH)
           
            img = cv2.resize(img, (RESIZE,RESIZE))
           
            IMG.append(np.array(img))
    return IMG

Mild_train = np.array(Dataset_loader('E:\\datasetfolder\\diabetic\\train\\Mild',224))
Moderate_train = np.array(Dataset_loader('E:\\datasetfolder\\diabetic\\train\\Moderate',224))
No_DR_train = np.array(Dataset_loader('E:\\datasetfolder\\diabetic\\train\\No_DR',224))
Proliferate_DR_train = np.array(Dataset_loader('E:\\datasetfolder\\diabetic\\train\\Proliferate_DR',224))
Severe_train = np.array(Dataset_loader('E:\\datasetfolder\\diabetic\\train\\Severe',224))

Mild_test = np.array(Dataset_loader('E:\\datasetfolder\\diabetic\\test\\Mild',224))
Moderate_test = np.array(Dataset_loader('E:\\datasetfolder\\diabetic\\test\\Moderate',224))
No_DR_test = np.array(Dataset_loader('E:\\datasetfolder\\diabetic\\test\\No_DR',224))
Proliferate_DR_test = np.array(Dataset_loader('E:\\datasetfolder\\diabetic\\test\\Proliferate_DR',224))
Severe_test = np.array(Dataset_loader('E:\\datasetfolder\\diabetic\\test\\Severe',224))

# Create Label

In [ ]:
#covid detection: covid,normal,pneumonia
# Create labels
Mild_train_label = np.zeros(len(Mild_train))
Moderate_train_label = np.ones(len(Moderate_train))
No_DR_train_label = np.zeros(len(No_DR_train))
Proliferate_DR_train_label = np.ones(len(Proliferate_DR_train))
Severe_train_label = np.zeros(len(Severe_train))

Mild_test_label = np.zeros(len(Mild_test))
Moderate_test_label = np.ones(len(Moderate_test))
No_DR_test_label = np.zeros(len(No_DR_test))
Proliferate_DR_test_label = np.ones(len(Proliferate_DR_test))
Severe_DR_test_label = np.zeros(len(Severe_test))

# Merge data 
X_train = np.concatenate((Mild_train, Moderate_train, No_DR_train,Proliferate_DR_train,Severe_train))
Y_train = np.concatenate((Mild_train_label, Moderate_train_label, Proliferate_DR_train_label,Severe_train_label))
X_test = np.concatenate((Mild_test, Moderate_test,No_DR_test,Proliferate_DR_test,Severe_test ))
Y_test = np.concatenate((Mild_test_label, Moderate_test_label, No_DR_test_label,Proliferate_DR_test_label,Severe_DR_test_label))

# Shuffle train data
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
Y_train = Y_train[s]

# Shuffle test data
s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
Y_test = Y_test[s]

# To categorical
Y_train = to_categorical(Y_train, num_classes= 5)
Y_test = to_categorical(Y_test, num_classes= 5)

# Train and Evalutation split

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    X_train, Y_train, 
    test_size=0.2, 
    random_state=11
)

# Display Some Images

In [ ]:
# # Display first 15 images of moles, and how they are classified
w=60
h=40
fig=plt.figure(figsize=(15, 15))
columns = 4
rows = 3

for i in range(1, columns*rows +1):
    ax = fig.add_subplot(rows, columns, i)
    if np.argmax(Y_train[i]) == 0:
        ax.title.set_text('Mild')
    elif np.argmax(Y_train[i]) == 1:
        ax.title.set_text('Moderate')
    elif np.argmax(Y_train[i]) == 2:
        ax.title.set_text('No_DR')
    elif np.argmax(Y_train[i]) == 3:
        ax.title.set_text('Proliferate_DR')
    elif np.argmax(Y_train[i]) == 4:
        ax.title.set_text('Severe')
    
    plt.imshow(x_train[i], interpolation='nearest')
plt.show()

# Data Generator

In [ ]:
BATCH_SIZE = 16

# Using original generator
train_generator = ImageDataGenerator(
        zoom_range=2,  # set range for random zoom
        rotation_range = 90,
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
    )

# Vgg 16 ----->

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, Activation, Dropout
from keras.models import Model
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from keras.models import Sequential
import keras  
from keras.applications.vgg16 import VGG16
#from  keras.applications import VGG16, ResNet50
from keras import backend as K
from keras import optimizers
import os
import numpy as np
import errno
from matplotlib import pyplot as plt
import time

In [ ]:
# test >> yes = 62 | no - 34
# training >> yes =  145 | no - 81

# NN Parameters
image_size = 224     
train_batchsize = 10  
epochs = 90

In [ ]:
# Image Dataset Directory
train_dir = 'C:\\Users\\Meghana\\Desktop\\jupytercode\\covid\\Data\\train' #'dataset/train'

def make_sure_path_exists(path):
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EXIST:
            raise

In [ ]:
# Train datagenerator
BATCH_SIZE = 16

# Using original generator
train_generator = ImageDataGenerator(
        zoom_range=2,  # set range for random zoom
        rotation_range = 90,
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
    )


In [ ]:
def vgg16_finetuned():

  vgg_conv = VGG16(weights='imagenet',
            include_top=False,
            input_shape=(image_size, image_size, 3))

  for layer in vgg_conv.layers[:-2]:
    layer.trainable = False
  lr=1e-3
  model = Sequential()
  model.add(vgg_conv)
  model.add(Flatten())
  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(5, activation= 'softmax'))
    
  model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=lr),
        metrics=['accuracy']
    )
       

  return model

In [ ]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 90:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 60:
        lr *= 1e-2
    elif epoch > 40:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:
K.clear_session()
gc.collect()

resnet = DenseNet201(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)


 


model = vgg16_finetuned()       #(resnet ,lr = 1e-4)
model.summary()

In [ ]:
# Learning Rate Reducer
learn_control = ReduceLROnPlateau(monitor='val_acc', patience=5,
                                  verbose=1,factor=0.2, min_lr=1e-7)

# Checkpoint
filepath="C:\\Users\\Meghana\\Desktop\\jupytercode\\covid\\Data\\vgg.weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

In [ ]:
history = model.fit_generator(
    train_generator.flow(x_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=x_train.shape[0] / BATCH_SIZE,
    epochs=15,
    validation_data=(x_val, y_val),
    callbacks=[learn_control, checkpoint]
)

**PREDICTION**


In [ ]:
model.load_weights("C:\\Users\\Meghana\\Desktop\\jupytercode\\covid\\Data\\vgg.weights.best.hdf5")

In [ ]:
Y_val_pred = model.predict(x_val)

In [ ]:
accuracy_score(np.argmax(y_val, axis=1), np.argmax(Y_val_pred, axis=1))

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
tta_steps = 10
predictions = []

for i in tqdm(range(tta_steps)):
    preds = model.predict_generator(train_generator.flow(X_test, batch_size=BATCH_SIZE, shuffle=False),
                                    steps = len(X_test)/BATCH_SIZE)
    
    predictions.append(preds)
    gc.collect()
    
Y_pred_tta = np.mean(predictions, axis=0)

In [ ]:
**TESTING**

In [ ]:
i=0
prop_class=[]
mis_class=[]

for i in range(len(Y_test)):
    if(np.argmax(Y_test[i])==np.argmax(Y_pred_tta[i])):
        prop_class.append(i)
    if(len(prop_class)==8):
        break

i=0
for i in range(len(Y_test)):
    if(not np.argmax(Y_test[i])==np.argmax(Y_pred_tta[i])):
        mis_class.append(i)
    if(len(mis_class)==8):
        break

# # Display first 8 images of benign
w=60
h=40
fig=plt.figure(figsize=(18, 10))
columns = 4
rows = 2

def Transfername(namecode):
    if namecode==0:
        return "Mild"
    elif namecode==1:
        return "Moderate"
    elif namecode==2:
        return "No_DR"
    elif namecode==3:
        return "Proliferate_DR"
    elif namecode==4:
        return "Severe"
    
for i in range(len(prop_class)):
    ax = fig.add_subplot(rows, columns, i+1)
    ax.set_title("Predicted result:"+ Transfername(np.argmax(Y_pred_tta[prop_class[i]]))
                       +"\n"+"Actual result: "+ Transfername(np.argmax(Y_test[prop_class[i]])))
    plt.imshow(X_test[prop_class[i]], interpolation='nearest')
plt.show()